In [1]:
#!pip install -r requirements.txt

In [1]:
from dotenv import load_dotenv
import os
from os import walk
from zipfile import ZipFile
import re
import pandas as pd
import sqlite3
import sqlalchemy
import pprint
import csv

In [2]:
env_file = "optiver-kaggle.env"
env_user_message = 'Please make sure that your file contains:\n\tKAGGLE_USERNAME="username"\n\tKAGGLE_KEY="xxxxxxxxxxxxxx"'
if os.path.isfile(env_file):
    print(f"{env_file} file already exists")
    print(env_user_message)
else:
    print(f"creating {env_file} file")
    !touch optiver-kaggle.env
    print(f"Created {env_file}")
    print(env_user_message)


optiver-kaggle.env file already exists
Please make sure that your file contains:
	KAGGLE_USERNAME="username"
	KAGGLE_KEY="xxxxxxxxxxxxxx"


In [3]:
load_dotenv(env_file)

True

In [4]:
def source_download(extraction_path):
    file_name = "optiver-realized-volatility-prediction.zip"
    !kaggle competitions download -c optiver-realized-volatility-prediction
    with ZipFile(file_name, 'r') as zip:
        print('Extracting all the files now...')
        zip.extractall(extraction_path)
        print('Done!')
    !rm -rf optiver-realized-volatility-prediction.zip

In [5]:
data_path = "./data"
download_message = "Downloading data files..."
if os.path.isdir(data_path):
    print(f"{data_path} directory already exists")
    if len(os.listdir(data_path)) == 0:
        print(f"{data_path} directory is empty.")
        print(download_message)
        source_download(data_path)
else:
    print(f"Creating {data_path} directory...")
    !mkdir data
    print(download_message)
    source_download(data_path)

./data directory already exists


In [6]:
engine = sqlalchemy.create_engine("sqlite:///OptiverKaggle.db")
conn = engine.connect()

In [7]:
g = globals()

In [8]:
def get_paths(root_path):
    paths = []
    for (dirpath, dirnames, filenames) in walk(root_path):
        if os.name == 'posix':
            if "/stock_id" in dirpath:
                paths.append(f"{dirpath}/{filenames[0]}")
        if os.name == 'nt':
            if "\stock_id" in dirpath:
                paths.append(f"{dirpath}\\{filenames[0]}")
            
    return paths

In [9]:
data_names = ["book_test", "book_train", "trade_test", "trade_train"]

In [10]:
def get_stock_id(path_substring):
    if os.name == 'posix':
        return re.sub('/[a-z0-9]*','', path_substring.replace('=','_'))
    if os.name == 'nt':
        return re.sub('\\\\[a-z0-9]*','', path_substring.replace('=','_'))

def get_df_name(dirname, stock_id):
    return f"{dirname}_{stock_id}"

def load_df(file_path):
    return pd.read_parquet(file_path, engine="pyarrow")

def read_data(paths):
    df_names = []
    for path in paths:
        if data_names[0] in path:
            stock_id = get_stock_id(path[25:38])
            df_name = get_df_name(data_names[0], stock_id)
            df_names.append(df_name)
            g[df_name] = load_df(path)
        elif data_names[1] in path:
            stock_id = get_stock_id(path[26:38])
            df_name = get_df_name(data_names[1], stock_id)
            df_names.append(df_name)
            g[df_name] = load_df(path)
        elif data_names[2] in path:
            stock_id = get_stock_id(path[26:38])
            df_name = get_df_name(data_names[2], stock_id)
            df_names.append(df_name)
            g[df_name] = load_df(path)
        elif data_names[3] in path:
            stock_id = get_stock_id(path[27:40])
            df_name = get_df_name(data_names[3], stock_id)
            df_names.append(df_name)
            g[df_name] = load_df(path)
    return df_names

In [11]:
def get_data(df_name):
     return g.get(df_name)

In [12]:
def seed_db(data_list):
    for data in data_list:
        if not engine.dialect.has_table(conn, data):
            get_data(data).to_sql(data, engine, if_exists='fail')
            print(f'{data} has been added to {conn.engine.url}')
        else:
            print(f'{data} already exists in {conn.engine.url}')

In [13]:
paths = get_paths(data_path)

In [14]:
data_list = read_data(paths)

In [15]:
# To preview the data in the DataFrames
# use the get_data() function to retive 
# the value from the global. Example below:

    # get_data(data_list[1])

In [16]:
#seed_db(data_list)

In [17]:
columns={}
for i in range(len(data_list)):
    signature = ", ".join(list(get_data(data_list[i]).columns))
    if signature not in columns:
        columns[signature] = 1
    else:
        columns[signature] += 1

In [18]:
book_columns = f"stock_id, {list(columns.keys())[0]}".split(",")
trade_columns = f"stock_id, {list(columns.keys())[1]}".split(",")

In [19]:
def data_to_csv():
    for name in data_names:
        if "trade" in name:
            with open(f"./data/{name}.csv", "w") as csvfile:
                writer = csv.writer(csvfile) 
                writer.writerow(trade_columns) 
        if "book" in name:
            with open(f"./data/{name}.csv", "w") as csvfile: 
                writer = csv.writer(csvfile) 
                writer.writerow(book_columns) 
        
    for data in data_list:
        if data_names[0] in data:
            get_data(data).insert(loc=0, column="stock_id", value=data[19:])
            get_data(data).to_csv(f"./data/{data_names[0]}.csv", mode="a", header=False)
        if data_names[1] in data:
            get_data(data).insert(loc=0, column="stock_id", value=data[20:])
            get_data(data).to_csv(f"./data/{data_names[1]}.csv", mode="a", header=False)
        if data_names[2] in data:
            get_data(data).insert(loc=0, column="stock_id", value=data[20:])
            get_data(data).to_csv(f"./data/{data_names[2]}.csv", mode="a", header=False)
        if data_names[3] in data:
            get_data(data).insert(loc=0, column="stock_id", value=data[21:])
            get_data(data).to_csv(f"./data/{data_names[3]}.csv", mode="a", header=False)

In [20]:
data_to_csv()